# TF-IDF 기반 피처 벡터 생성 - 네이버 영화 리뷰
* TF-IDF는 토큰화 이전의 문장 형태의 데이터를 사용

### Cleaning된 문장 데이터를 로딩하여 사용

In [1]:
import pandas as pd
train_data = pd.read_csv('DataSet_NaverMovie/train_data_prep_sent.csv')
test_data = pd.read_csv('DataSet_NaverMovie/test_data_prep_sent.csv')
print('훈련 샘플의 개수 :',len(train_data)) # 훈련용 리뷰 개수 출력
print('테스트 샘플의 개수 :',len(test_data)) # 테스트용 리뷰 개수 출력

훈련 샘플의 개수 : 145393
테스트 샘플의 개수 : 48852


In [2]:
train_data.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


In [ ]:
X_train = list(train_data['tokenized'])
X_test = list(test_data['tokenized'])

import numpy as np
y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])

In [ ]:
X_train[:3]

In [ ]:
len(X_train), len(y_train), len(X_test), len(y_test)

# TF-IDF 기반 피처 벡터 생성 
* sklearn 라이브러리의 TfidfVectorizer로 학습하여 Feature Vectore 생성 모델 구축
   - from sklearn.feature_extraction.text import TfidfVectorizer
   - 모델 생성 : *model* = **TfidfVectorizer**(tokenizer=*tokenizer_func*, *optional parameters*)
   - 모델 학습 : *model*.**fit**(*train_text_data*)
   - Feature Vertor로 변환 : *feature_vector* = *model*.**transform**(*text_data*)
* 실행시간 10분 정도 걸림

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [5]:
# Tokenizer 정의
from konlpy.tag import Okt
okt = Okt()
def okt_tokenizer(text):
    #tokens = okt.morphs(text, stem=True)
    tokens = okt.morphs(text)
    return tokens

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_model = TfidfVectorizer(tokenizer=okt_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_model.fit(train_data['document'])
train_tfidf = tfidf.transform(train_data['document'])

KeyboardInterrupt: 

In [4]:
# 불용어 정의
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words=stopwords, tokenizer=okt_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf.fit(train_data['document'])
train_tfidf = tfidf.transform(train_data['document'])

C:\Users\worki\AppData\Roaming\Python\Python311\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\worki\AppData\Roaming\Python\Python311\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['가다', '늘다', '들다', '오다', '자다'] not in stop_words.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
train_tfidf

In [ ]:
import joblib
joblib.dump(tfidf_model, 'tfidf_model.pkl')

## 2-2) 감성 분류(Sentiment Analysis) 모델 구축

* 로지스틱 회귀를 이용한 이진 분류

1. 로지스틱 회귀 기반 분석 모델 학습 (sklearn.linear_model.LogisticRegression)
1. 최적의 하이퍼파라미터 갖는 학습 모델 구축 (sklearn.model_selection.**GridSearchCV**)
1. 최적 학습 모델에 대한 평가

### 2-2-1. 로지스틱 회귀 기반 분석 모델 학습

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [ ]:
from sklearn.linear_model import LogisticRegression

SA_LogR = LogisticRegression(random_state = 0)

In [ ]:
SA_LogR.fit(train_tfidf, train_df['label'])

### 2-2-2. 로지스틱 회귀의  최적의 하이퍼파라미터 찾기

* 로지스틱 회귀의 하이퍼파라미터 C의 최적값 찾기

1. 최적의 하이퍼파라미터 찾는 객체 생성 : *gridSearchCVobject* = **GridSearchCV**(*estimater*, *param_grid*, *optional parameters*)
1. 학습 데이터로 최적의 하이퍼파라미터 도출 : *gridSearchCVobject*.**fit**(*train_data*)
1. 최적의 하이퍼파라미터를 적용한 학습 모델 저장 : *best_model* = *gridSearchCVobject*.**best_estimator_**

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {'C': [1, 3, 3.5, 4, 4.5, 5]}
SA_LogR_GridCV = GridSearchCV(SA_lr, param_grid=params, cv=3, scoring='accuracy', verbose=1)

In [ ]:
SA_LogR_GridCV.fit(train_tfidf, train_df['label'])

In [ ]:
print(SA_LogR_GridCV.best_params_, round(SA_LogR_GridCV.best_score_, 4))

### 2-2-3. 최적 분석 모델 저장

In [ ]:
# 최적 파라미터의 best 모델 저장
SA_LogR_Best = SA_LogR_GridCV.best_estimator_

# 3. 분석 모델 평가

## 3-1. 평가용 데이터를 이용하여 감성 분석 모델 정확도

In [ ]:
# 평가용 데이터의 피처 벡터화 : 실행시간 6분 정도 걸립니다 ☺
test_tfidf = tfidf.transform(test_df['document'])

In [ ]:
test_predict = SA_LogR_Best.predict(test_tfidf)

In [ ]:
from sklearn.metrics import accuracy_score

print('감성 분석 정확도 : ', round(accuracy_score(test_df['label'], test_predict), 3))

## 3-2. 새로운 텍스트에 대한 감성 예측
1. 문장 입력 받기
2. 입력 문장 전처리
3. Feature Vector 생성
4. 감성 분석 모델 적용 -> 감성 분석
5. 예측값 출력

In [ ]:
in_sent = input('감성 분석할 문장입력 >> ')

In [ ]:
# 0) 입력 텍스트에 대한 전처리 수행
in_sent = re.compile(r'[ㄱ-ㅣ가-힣]+').findall(in_sent)
print(in_sent)
in_sent = [" ".join(in_sent)]
print(in_sent)

In [ ]:
# 1) 입력 텍스트의 피처 벡터화
st_tfidf = tfidf.transform(in_sent)

# 2) 최적 감성분석 모델에 적용하여 감성 분석 평가
st_predict = SA_LogR_Best.predict(st_tfidf)

In [ ]:
# 3) 예측 값 출력하기
if(st_predict== 0):
    print(in_sent , "->> 부정 감성")
else :
    print(in_sent , "->> 긍정 감성")

### 검색어로 crawling한 기사에 대한 긍정, 부정 현황 파악
1. 데이터 로딩 (파이썬 객체로)
1. 입력 문장 전처리
1. Feature Vector 생성
1. 감성 분석 모델 적용 -> 감성 분석
1. 예측값 출력

In [ ]:
import joblib

joblib.dump(tfidf_model, 'tfidf_model.pkl')
joblib.dump(SA_LogR_model, 'SA_LogR_model.pkl')